<a href="https://colab.research.google.com/github/Akshat-Math/Mnist-Classification-using-CNN/blob/main/CNN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from keras.datasets import mnist

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
#one hot encoding
n_classes = 10
y_train = tf.keras.utils.to_categorical(y_train,n_classes)
y_test = tf.keras.utils.to_categorical(y_test,n_classes)

In [ ]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784 #28x28

convo_layer_1 = 32  #32 units
stride_convo1 = 1
convo_layer_2 = 64  #64 units
stride_convo2 = 1
#5x5 filters for both convolutional layers
convo_1_k = 5
convo_2_k =5
#2x2 filters for both max_pool layers
max_pool_k1 = 2
max_pool_k2 = 2

n_hidden = 1024
n_output = 10

input_size_to_hidden = 7 * 7 * (convo_layer_2)

In [ ]:
x_train = x_train.reshape(-1, input_height, input_width, input_channels)
x_test = x_test.reshape(-1, input_height, input_width, input_channels)

In [ ]:
weights = { 'wc1' : tf.Variable(tf.random.normal([convo_1_k , convo_1_k , input_channels, convo_layer_1])), #shape of weights for convo layer 1  = 5x5x32x1
            'wc2': tf.Variable(tf.random.normal([convo_2_k, convo_2_k, convo_layer_1, convo_layer_2])), #shape of weights for convo layer 1 = 5x5x64x32
            'whl' : tf.Variable(tf.random.normal([input_size_to_hidden,n_hidden])), #shape = 7x7x64x1024 = 3136x64
            'wop' : tf.Variable(tf.random.normal([n_hidden,n_output]))  #shape = 1024x10
          }

biases = { 'b1' : tf.Variable(tf.random.normal([convo_layer_1])), #biases for every unit of convo 1 = no. of units = 32
           'b2' : tf.Variable(tf.random.normal([convo_layer_2])),
           'hl' : tf.Variable(tf.random.normal([n_hidden])),
           'op' : tf.Variable(tf.random.normal([n_output])),
         }

In [ ]:
def conv(x,weights,bias,strides = 1):
  #strides ka shape should be same as that of x, so we dont want to skip any image so we kept 1 in starting that is compared to batch size
  #height and weight ki jagah we have strides
  #for input_channels ki jagah means the depth of filter is 1 cause ek pura block of filter is fitted
  out = tf.nn.conv2d(x , weights, padding = "SAME" , strides=[1,strides,strides,1])  #applying filter to convo_layer_1
  out = tf.nn.bias_add(out,bias)
  out = tf.nn.relu(out)

  return out


In [ ]:
def maxpooling(x, k=2):
  #ksize = no.of_images x height x width x no.of_channels
  return tf.nn.max_pool(x, padding = "SAME" , ksize = [1,k,k,1] , strides = [1,k,k,1])

In [ ]:
def cnn(x,weights,biases,keep_prob):
  #we need to reshape our input first
  #'batch_sizex28x28x1
  #batch size so that those all images are reshaped which are in batch
  #here -1 says ki mai khud pata laga lunga so wo batch_size khud pata lega
   x = tf.reshape(x,shape = (-1,input_height,input_width,input_channels))

   convo_layer1_op = conv( x , weights['wc1'] , biases['b1'], stride_convo1)  #output of convolutional layer 1
   conv1_pool_op = maxpooling(convo_layer1_op, max_pool_k1) #output of maxpool layer 1

   convo_layer2_op = conv( conv1_pool_op,weights['wc2'] , biases['b2'], stride_convo2) #output of convolutional layer 2
   conv2_pool_op = maxpooling(convo_layer2_op, max_pool_k2) #output of maxpool layer 2

   #reshaping before dense layer
   hidden_input = tf.reshape(conv2_pool_op, shape = [-1,input_size_to_hidden] )
   hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['whl']),biases['hl'])
   hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
   #dropout layer
   hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
   output = tf.add(tf.matmul(hidden_output,weights['wop']),biases['op'])

   return output

In [ ]:
x = tf.compat.v1.placeholder('float32', [None,input_height,input_width,input_channels])  #None represents that we dont know ki how many no. of images will we pass but we know that the features will be 784
print(x.shape)
y = tf.compat.v1.placeholder('float32',[None,n_output]) #this will be of shape no. of images
print(y.shape)
keep_prob = tf.compat.v1.placeholder("float32")

(None, 28, 28, 1)
(None, 10)


In [ ]:
pred = cnn(x,weights,biases,keep_prob)

In [ ]:
#cost fxn, here we use cross entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y)) #logits means predictions here

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [ ]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
batch_size = 100
num_batches = len(x_train) // batch_size

for i in range(25):
    total_cost = 0
    for j in range(num_batches):
        start_idx = j * batch_size
        end_idx = (j + 1) * batch_size
        batch_x, batch_y = x_train[start_idx:end_idx], y_train[start_idx:end_idx]
        c, _ = sess.run([cost, optimize], feed_dict={x: batch_x, y: batch_y, keep_prob: 0.8})
        total_cost += c
    print(total_cost)

319704837.5136719
1844154.7660565376
199939.3951101303
44181.9602060318
38093.263711333275
18837.738605737686
17716.1618309021
11934.943495035172
5725.7557628154755
5776.500092744827
11326.775836706161
20164.10687494278
103474.6178996563
76069.18829917908
5038.784490346909
2234.8819806575775
26346.89414024353
164275.21172523499
18451.07860517502
1380.9149661064148
1380.9356665611267
1380.8677780628204
1460.7347548007965
1380.8235375881195
1380.936760187149


In [ ]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:x_train,
                                              y:y_train, keep_prob:1.0})
correct_preds.sum()

6265